In [ ]:
#@title GPU information
!nvidia-smi

In [ ]:
#@title Download Blender
import os
import shutil
from google.colab import files, drive

blender_version = '2.92' #@param ['2.79b', '2.80rc3', '2.81a', '2.82a', '2.83.12', '2.90.1', '2.91.2', '2.92'] {allow-input: false}
blend_file_path = 'path/to/file.blend' #@param {type: 'string'}

!rm -r output
!rm -r render
!rm -r export
!rm *.blend

blender_url_dict = {'2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
                    '2.80rc3' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.80/blender-2.80rc3-linux-glibc217-x86_64.tar.bz2",
                    '2.81a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.81/blender-2.81a-linux-glibc217-x86_64.tar.bz2",
                    '2.82a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.82/blender-2.82a-linux64.tar.xz",
                    '2.83.12' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.83/blender-2.83.12-linux64.tar.xz",
                    '2.90.1'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.90/blender-2.90.1-linux64.tar.xz",
                    '2.91.2'  : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.91/blender-2.91.2-linux64.tar.xz",
                    '2.92'    : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.92/blender-2.92.0-linux64.tar.xz"}

blender_url = blender_url_dict[blender_version]
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

In [ ]:
#@title Upload Blend file

upload_type = 'google_drive' #@param ['direct', 'google_drive', 'url', 'gdrive_relative'] {allow-input: false}
drive_path = 'Blender/' #@param {type: 'string'}
url_blend = '' #@param {type: 'string'}

download_type = 'direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}


uploaded_filename = ""

#if upload_type == 'google_drive' or upload_type == 'gdrive_relative' or download_type == 'google_drive' or download_type == 'gdrive_direct':
if upload_type == 'google_drive' or upload_type == 'gdrive_relative':
    drive.mount('/drive')

if upload_type == 'direct':
    uploaded = files.upload()
    for fn in uploaded.keys():
        uploaded_filename = fn
elif upload_type == 'url':
    !wget -nc $url_blend
    uploaded_filename = os.path.basename(url_blend)
elif upload_type == 'google_drive':
    shutil.copy('/drive/My Drive/' + drive_path, '.')
    uploaded_filename = os.path.basename(drive_path)

!rm -r render
!mkdir render

if upload_type == 'gdrive_relative':
    if not drive_path.endswith('/'):
        drive_path += '/'
    !cp -r '/drive/My Drive/{drive_path}.' 'render/'
elif uploaded_filename.lower().endswith('.zip'):
    !unzip -o $uploaded_filename -d 'render/'
elif uploaded_filename.lower().endswith('.blend'):
    shutil.copy(uploaded_filename, 'render/')
    blend_file_path = uploaded_filename
else:
    raise SystemExit("Invalid file extension, only .blend and .zip can be uploaded.")



In [ ]:
#@title Scene setup:

Samples_Count = 128 #@param {type:"integer"}
resolution_x = 1920 #@param {type:"integer"}
resolution_y = 1080 #@param {type:"integer"}
scale = 1 #@param {type:"raw"}
adaptive_threshold = 0.1 #@param {type:"raw"}
use_adaptive_sampling = False #@param {type:"boolean"}
denoiser = 'OPENIMAGEDENOISE' #@param ['OPENIMAGEDENOISE',"NLM", "OPTIX"]
use_denoising = False #@param {type:"boolean"}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}

#@markdown ---

whatToRender = 'renderOneFrame' #@param ["renderOneFrame", "renderAllFrame", "renderWithinRange"] {allow-input: false}
#@markdown StartFrame = the frame you want to start with or the frame number for the renderOneFrame option
startFrame =  1#@param {type: "integer"}
endFrame =  250#@param {type: "integer"}

!rm setgpu.py

data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "bpy.context.scene.cycles.samples = "+str(Samples_Count)+"\n"+\
    "bpy.context.scene.render.resolution_x = "+str(resolution_x * scale)+"\n"+\
    "bpy.context.scene.render.resolution_y = "+str(resolution_y * scale)+"\n"+\
    "bpy.context.scene.cycles.adaptive_threshold = "+str(adaptive_threshold)+"\n"+\
    "bpy.context.scene.cycles.use_adaptive_sampling = "+str(use_adaptive_sampling)+"\n"+\
    "bpy.context.scene.cycles.denoiser = '" +str(denoiser)+ "'\n"+\
    "bpy.context.scene.cycles.use_denoising = "+str(use_denoising)+"\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for scene in bpy.data.scenes:\n"+\
    "    scene.render.tile_x = 256\n"+\
    "    scene.render.tile_y = 256\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

#!rm -r output
!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

if download_type != 'gdrive_direct':
    output_path = 'output/' + output_name
else:
    output_path = '/drive/My Drive/' + drive_output_path + output_name


if whatToRender == 'renderOneFrame':
  !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $startFrame
elif whatToRender == 'renderAllFrame':
  !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -a
elif whatToRender == 'renderWithinRange':
  !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -s $startFrame -e $endFrame -a



In [ ]:
#@title Download

output = 'render/export/' #@param ['output',"render/export/Instagram", "render/export/"]

path, dirs, files_folder = next(os.walk(output))

print(*files_folder)
output_folder_name = output_name.replace('#', '') + 'render'

if download_type == 'gdrive_direct':
    pass
elif len(files_folder) == 1:
    render_img = output + '/' + files_folder[0]
    if download_type == 'direct':
        files.download(output + '/' + files_folder[0])
    else:
        shutil.copy('/content/' + render_img, '/drive/My Drive/' + drive_output_path)
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', output + '/')
    if download_type == 'direct':
        files.download(output_folder_name + '.zip')
    else:
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/My Drive/' + drive_output_path)
elif download_type == 'direct':
    for f in files_folder:
        files.download(output + f"/{f}")
    # Drive, no zip
    else:
        for f in files_folder:
          shutil.copy("/content/output/" + f, '/drive/My Drive/' + drive_output_path + f)
else:
    raise SystemExit("No frames are rendered.")